In [ ]:
import pandas as pd
import numpy as np
import glob
from numpy.linalg import norm
import re
from scipy.spatial.distance import euclidean
import sys
import matplotlib.pyplot as plt
sys.path.insert(0,'../')
from OrderNpersist import to_vecs
import time

In [ ]:
def handle_boundaries(cell_track,pr = False):
    # look for boundary crossings in any
    # of the coordinates
    cell_track2 = cell_track.copy()
    for i in range(len(cell_track) - 1):
        dif = np.subtract(cell_track[i],cell_track[i+1])
        for j,coordinate in enumerate(dif):
            if coordinate > 32:
                # went over boundary from 64 -> 0
                cell_track2[:i + 1,j] -= 64

            elif coordinate < -32:
                # form 0 -> 64

                cell_track2[:i + 1,j] += 64
    return cell_track2

In [ ]:
def build_csv(path):
    """ Path is folder containing all celltracks of all max-lambda combinations.
    """
    # find unique param pairs in folder :
    num_ptrn = '[-+]?\d*\.\d+|\d+'
    gtypes = ['ER','BA', 'WS', 'PW','GM']
    folder = glob.glob(path)
    files = glob.glob(path+ '/*')
    params = set([s.split(re.findall(num_ptrn,s)[-2])[-1] for s in files])
    params = {p for p in params if p != '.txt'}
    print(params)
    ind_rows = []
    global_rows = []
    for gt in params:
        t1 = time.time()
        files = glob.glob(path+ '/*' + gt)
        #print(files)
        print(gt)
        Type = gt[:2]
        itr = gt[2]
        print(Type,itr)
        files.sort(reverse = True,key = lambda x: int(re.findall(num_ptrn,x)[-1]))
        # extract tracks from files :
        tracks = [np.loadtxt(f) for f in files]
        print('unfiltered cells ',len(tracks))
        tracks = np.array([t for t in tracks if len(t) == 200])
        #tracks2 = [handle_boundaries(tr) for tr in tracks]
        vec_tracks = np.array([to_vecs(handle_boundaries(x)) for x in tracks])
        return tracks,vec_tracks

tracks,vec_tracks = build_csv('/home/lau/Desktop/Thesis Stuff/test_tracks')

In [ ]:
print(tracks.shape,vec_tracks.shape)

In [ ]:
for i,v in enumerate(vec_tracks[:,150]):
    if norm(v) > 7:
        print(i,norm(v))

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
def plot_track(cell_track):
    x = [c[0] for c in cell_track]
    y = [c[1] for c in cell_track]
    z = [c[2] for c in cell_track]

    #fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot3D(x,y,z)
    ax.scatter3D(x[0],y[0],z[0],label = 'start')
    ax.scatter3D(x[-1],y[-1],z[-1],label = 'end')
    ax.legend()
plot_track(handle_boundaries(tracks[201]))
for i,v in enumerate(to_vecs(handle_boundaries(tracks[201]))):
    print(i,norm(v),norm(to_vecs(tracks[201])[i]))
    print(tracks[201,i],tracks[201,i+1])

In [ ]:
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
def plot_track(cell_track):
    x = [c[0] for c in cell_track]
    y = [c[1] for c in cell_track]
    z = [c[2] for c in cell_track]

    #fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot3D(x,y,z)
    ax.scatter3D(x[0],y[0],z[0],label = 'start')
    ax.scatter3D(x[-1],y[-1],z[-1],label = 'end')
    ax.legend()
plot_track(tracks[201])

In [ ]:
# vector field at time 0 : 

tracks_at_zero = tracks[:,0]
vecs_at_zero = vec_tracks[:,0]


import plotly.graph_objects as go

fig = go.Figure(data=go.Cone(
    x=tracks_at_zero[:,0],
    y=tracks_at_zero[:,1],
    z=tracks_at_zero[:,2],
    u=vecs_at_zero[:,0],
    v=vecs_at_zero[:,1],
    w=vecs_at_zero[:,2],
    sizemode="absolute",
    sizeref=10,
    anchor="tip"))

fig.update_layout(
      scene=dict(domain_x=[0, 1],
                 camera_eye=dict(x=-1.57, y=1.36, z=0.58)))

fig.show()

In [ ]:
# animate : 

fig = go.Figure(
    data=[go.Cone(
    x=tracks_at_zero[:,0],
    y=tracks_at_zero[:,1],
    z=tracks_at_zero[:,2],
    u=vecs_at_zero[:,0],
    v=vecs_at_zero[:,1],
    w=vecs_at_zero[:,2],
    sizemode="absolute",
    sizeref=10,
    anchor="tip")],
    layout=go.Layout(
#         xaxis=dict(range=[0,64], autorange=False),
#         yaxis=dict(range=[0,64], autorange=False),
#         zaxis=dict(range=[0,64], autorange=False),
        title="Start Title",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames = [go.Frame(data = [go.Cone(
    x=tracks[:,t,0],
    y=tracks[:,t,1],
    z=tracks[:,t,2],
    u=vec_tracks[:,t,0],
    v=vec_tracks[:,t,1],
    w=vec_tracks[:,t,2],
    sizemode="absolute",
    sizeref=10,
    anchor="tip")]) for t in range(198)]# + [layout=go.Layout(title_text="End Title")]
#     frames=[go.Frame(data=[go.Scatter(x=[1, 2], y=[1, 2])]),
#             go.Frame(data=[go.Scatter(x=[1, 4], y=[1, 4])]),
#             go.Frame(data=[go.Scatter(x=[3, 4], y=[3, 4])],
#                     ]
)

fig.show()